In [4]:
%load_ext rich
%load_ext autoreload
%autoreload 2

In [5]:
import polars as pl
import pointblank as pb

from odyssey.core import *
from odyssey.explore import *

c:\Users\00113294\Documents\WIP\Validating IPAQ\.venv\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [6]:
from config import INTERIM_DATA

In [7]:
g220 = Dataset("G220_Q.sav", INTERIM_DATA)
lf_g220, _meta = g220.load_data()
df_g220 = lf_g220.collect()

In [8]:
g222 = Dataset("G222_Q.sav", INTERIM_DATA)
lf_g222, _meta = g222.load_data()
df_g222 = lf_g222.collect()

In [9]:
from typing import Callable

def check_total_mins(
    hpd_column: str,
    mpd_column: str
    ) -> Callable:
    """
    Returns a preprocessing function to verify the minutes for a given category have been correctly calculated.
    Cap the total at 180 minutes, and preserve null values.
    """
    def preprocessor(df: pl.DataFrame) -> pl.DataFrame:
        return df.with_columns(
            (pl.col(hpd_column).fill_null(0) * 60 + pl.col(mpd_column).fill_null(0))
            .pipe(lambda expr: pl.when(expr > 180).then(180).otherwise(expr))
            .alias("check")
        )
    return preprocessor

def check_met(
    mins_column: str, 
    n_days_column: str, 
    met_column: str,
    factor: int|float # the corresponding factor for the activity (Vig: 8, Mod: 4, Walk: 3.3)
    ) -> Callable:
    """Returns a preprocessing function to verify the calculated MET value for a given category."""
    def preprocessor(df: pl.DataFrame) -> pl.DataFrame:
        return df.with_columns(
            (pl.col(mins_column).fill_null(0) * pl.col(n_days_column).fill_null(0) * factor).alias("check"),
            pl.col(met_column).fill_null(0)
        )
    return preprocessor

def check_tot_met(
    met_columns: list[str],
    tot_met_column: str
    ) -> Callable:
    """Returns a preprocessing function to verify the calculated total MET value."""
    def preprocessor(df: pl.DataFrame) -> pl.DataFrame:
        expr = sum(pl.col(col).fill_null(0) for col in met_columns)
        
        return df.with_columns(
            expr.alias("check"),
            pl.col(tot_met_column).fill_null(0) # Fill nulls with 0; otherwise the validation skips if one value in a comparison is null
        )
    return preprocessor

In [ ]:
#TODO: flesh this out and run validations after initial harmonisation

def check_ipaq_cat(
    vig_d: str, # days of vigorous exercise per week
    mod_d: str,
    walk_d: str,
    vig_m: str, # mins of vigorous exercise per day
    mod_m: str,
    walk_m: str,
    tot_met: str, # total MET minutes per week
    cat: str # IPAQ category (low: 0, moderate: 1, high: 2)
    ) -> Callable:
    """
    Returns a preprocessing function to verify the IPAQ category.

    HIGH: 2
    Vigorous exercise on 3+ days AND >= 1500 MET mins per week
    OR any exercise on 7 days AND >= 3000 MET mins per week

    MODERATE: 1
    Vig exercise 3+ days AND/OR walking 30 mins/day
    OR Mod exercise 5+ days AND/OR walking 30 mins/day
    OR any exercise on 5+ days AND >= 600 MET mins per week

    LOW: 0
    None of the above criteria
        """
    def preprocessor(df: pl.DataFrame) -> pl.DataFrame:
        return df.with_columns(
            # We don't have the data to clearly determine "exercise of any kind on 7 days"
            # Based on my assumption that means the participant exercised every day in the week
            # I've accounted it such that if the participant did two forms of exercise in a single day
            # That would be counted as if they exercised on two days
            (pl.when(
                (pl.col(vig_d).ge(3) & pl.col(tot_met).ge(1500)) | 
                (sum(pl.col(col).fill_null(0) for col in [vig_d, mod_d, walk_d]).ge(7) & pl.col(tot_met).ge(3000))
            ).then(2)
            # The "AND/OR walking 30 mins/day" is also unfortunately ambiguous
            .when(
                (pl.col(vig_d).ge(3) & pl.col(vig_m).ge(30)) |
                (pl.col(mod_d).ge(5) & pl.col(mod_m).ge(30)) |
                (sum(pl.col(col).fill_null(0) for col in [vig_d, mod_d, walk_d]).ge(5) & pl.col(tot_met).ge(600))
            ).then(1)
            .when(pl.col(tot_met).is_null()).then(None) # Don't impute 0 if data is missing
            .otherwise(0)
            ).alias("check"),
            pl.col(cat).fill_null(0) # Fill nulls with 0; otherwise the validation skips if one value in a comparison is null
        )
    return preprocessor

In [16]:
def test_validate_ipaq(
    prefix: str, # prefix for the dataset
    df: pl.DataFrame
    ) -> pb.Validate:

    validation = (
        pb.Validate(
            data=df,
        )
        .col_vals_eq(
            columns=f"{prefix}_IPAQ_CAT",
            value=pb.col("check"),
            pre=check_ipaq_cat(
                f"{prefix}_IPAQ_VIG_D", f"{prefix}_IPAQ_MOD_D", f"{prefix}_IPAQ_WALK_D",
                f"{prefix}_IPAQ_VIG_MINS", f"{prefix}_IPAQ_MOD_MINS", f"{prefix}_IPAQ_WALK_MINS",
                f"{prefix}_IPAQ_TOT_MET", f"{prefix}_IPAQ_CAT"
            ),
            brief="Check `IPAQ_CAT` is correctly calculated."
        )
    ).interrogate()

    return validation

In [18]:
v = test_validate_ipaq("G220", df_g220)
v

Validate(
    data=shape: (1_487, 978)
┌─────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ ID      ┆ G220_DNWN  ┆ G220_MQ_D ┆ G220_THIN ┆ … ┆ G220_DASS ┆ G220_DASS ┆ G220_DASS ┆ G220_DASS │
│ ---     ┆ ---        ┆ NWN       ┆ K_DNWN    ┆   ┆ _ANX_CAT  ┆ _STR_SCOR ┆ _STR_CAT  ┆ _TOT_SCOR │
│ f64     ┆ date       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ E         ┆ ---       ┆ E         │
│         ┆            ┆ date      ┆ date      ┆   ┆ f64       ┆ ---       ┆ f64       ┆ ---       │
│         ┆            ┆           ┆           ┆   ┆           ┆ f64       ┆           ┆ f64       │
╞═════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 10010.0 ┆ 2010-03-11 ┆ 2010-03-1 ┆ null      ┆ … ┆ 1.0       ┆ 12.0      ┆ 1.0       ┆ 30.0      │
│         ┆            ┆ 1         ┆           ┆   ┆           ┆           ┆           ┆           │
│ 10020.0 ┆ 2010-05-01 ┆ 2010-05-1 ┆ 2010-02-2 ┆ … ┆ 1.0       ┆ 4.0       ┆ 1.0       ┆ 10.0      │
│         ┆            ┆ 6         ┆ 7         ┆   ┆           ┆           ┆           ┆           │
│ 10030.0 ┆ 2011-04-04 ┆ 2011-04-0 ┆ null      ┆ … ┆ 2.0       ┆ 14.0      ┆ 1.0       ┆ 30.0      │
│         ┆            ┆ 4         ┆           ┆   ┆           ┆           ┆           ┆           │
│ 10040.0 ┆ 2010-05-12 ┆ 2010-04-1 ┆ 2010-03-0 ┆ … ┆ 1.0       ┆ 16.0      ┆ 2.0       ┆ 36.0      │
│         ┆            ┆ 2         ┆ 1         ┆   ┆           ┆           ┆           ┆           │
│ 10050.0 ┆ 2010-03-09 ┆ 2010-03-1 ┆ 2010-03-0 ┆ … ┆ 1.0       ┆ 2.0       ┆ 1.0       ┆ 14.0      │
│         ┆            ┆ 3         ┆ 2         ┆   ┆           ┆           ┆           ┆           │
│ …       ┆ …          ┆ …         ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …         │
│ 54540.0 ┆ null       ┆ null      ┆ 2011-12-1 ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│         ┆            ┆           ┆ 1         ┆   ┆           ┆           ┆           ┆           │
│ 54550.0 ┆ 2011-10-22 ┆ 2011-10-2 ┆ 2011-10-2 ┆ … ┆ 3.0       ┆ 12.0      ┆ 1.0       ┆ 26.0      │
│         ┆            ┆ 3         ┆ 0         ┆   ┆           ┆           ┆           ┆           │
│ 54580.0 ┆ 2011-09-29 ┆ 2011-09-2 ┆ 2011-09-2 ┆ … ┆ 1.0       ┆ 0.0       ┆ 1.0       ┆ 0.0       │
│         ┆            ┆ 9         ┆ 9         ┆   ┆           ┆           ┆           ┆           │
│ 54590.0 ┆ 2011-10-10 ┆ 2011-10-1 ┆ 2011-10-1 ┆ … ┆ 4.0       ┆ 26.0      ┆ 4.0       ┆ 80.0      │
│         ┆            ┆ 0         ┆ 0         ┆   ┆           ┆           ┆           ┆           │
│ 54602.0 ┆ 2011-09-13 ┆ 2011-09-1 ┆ 2011-09-1 ┆ … ┆ 3.0       ┆ 2.0       ┆ 1.0       ┆ 14.0      │
│         ┆            ┆ 3         ┆ 3         ┆   ┆           ┆           ┆           ┆           │
└─────────┴────────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴───────────┘,
    tbl_name=None,
    label=None,
    thresholds=Thresholds(warning=None, error=None, critical=None),
    actions=None,
    final_actions=None,
    brief=None,
    lang='en',
    locale='en'
)

In [20]:
v.get_step_report(i=1, columns_subset=pb.matches(r"ID|_MET|IPAQ_CAT"))

GT(
    _tbl_data=shape: (10, 7)
┌───────────┬─────────┬───────────────┬───────────────┬──────────────┬──────────────┬──────────────┐
│ _row_num_ ┆ ID      ┆ G220_IPAQ_VIG ┆ G220_IPAQ_MOD ┆ G220_IPAQ_WA ┆ G220_IPAQ_TO ┆ G220_IPAQ_CA │
│ ---       ┆ ---     ┆ _MET          ┆ _MET          ┆ LK_MET       ┆ T_MET        ┆ T            │
│ u32       ┆ f64     ┆ ---           ┆ ---           ┆ ---          ┆ ---          ┆ ---          │
│           ┆         ┆ f64           ┆ f64           ┆ f64          ┆ f64          ┆ f64          │
╞═══════════╪═════════╪═══════════════╪═══════════════╪══════════════╪══════════════╪══════════════╡
│ 3         ┆ 10030.0 ┆ 1920.0        ┆ 0.0           ┆ null         ┆ null         ┆ 0.0          │
│ 39        ┆ 10720.0 ┆ 7200.0        ┆ null          ┆ null         ┆ null         ┆ 0.0          │
│ 58        ┆ 11180.0 ┆ 3840.0        ┆ 2160.0        ┆ 4158.0       ┆ 10158.0      ┆ 0.0          │
│ 61        ┆ 11230.0 ┆ 10080.0       ┆ 2520.0        ┆ 4158.0       ┆ 16758.0      ┆ 0.0          │
│ 106       ┆ 12160.0 ┆ 2880.0        ┆ null          ┆ 4158.0       ┆ null         ┆ 0.0          │
│ 134       ┆ 12790.0 ┆ 8640.0        ┆ 5040.0        ┆ 4158.0       ┆ 17838.0      ┆ 0.0          │
│ 137       ┆ 12820.0 ┆ 0.0           ┆ 0.0           ┆ 495.0        ┆ 495.0        ┆ 1.0          │
│ 139       ┆ 12900.0 ┆ 7200.0        ┆ null          ┆ null         ┆ null         ┆ 0.0          │
│ 181       ┆ 13810.0 ┆ 1920.0        ┆ null          ┆ 1188.0       ┆ null         ┆ 0.0          │
│ 198       ┆ 14110.0 ┆ 3600.0        ┆ 0.0           ┆ null         ┆ null         ┆ 0.0          │
└───────────┴─────────┴───────────────┴───────────────┴──────────────┴──────────────┴──────────────┘,
    _body=<great_tables._gt_data.Body object at 0x00000168D13D00F0>,
    _boxhead=Boxhead([ColInfo(var='_row_num_', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='right', column_width='33.4px'), ColInfo(var='ID', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>ID</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Float64</em></div></div>"), column_align='right', column_width='65px'), ColInfo(var='G220_IPAQ_VIG_MET', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>G220_IPAQ_VIG_MET</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Float64</em></div></div>"), column_align='right', column_width='143px'), ColInfo(var='G220_IPAQ_MOD_MET', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>G220_IPAQ_MOD_MET</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Float64</em></div></div>"), column_align='right', column_width='143px'), ColInfo(var='G220_IPAQ_WALK_MET', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>G220_IPAQ_WALK_MET</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Float64</em></div></div>"), column_align='right', column_width='150px'), ColInfo(var='G220_IPAQ_TOT_MET', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text="<div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; margin-bottom: 2px;'>G220_IPAQ_TOT_MET</div><div style='white-space: nowrap; text-overflow: ellipsis; overflow: hidden; padding-top: 2px; margin-top: 2px;'><em>Fl

In [ ]:
def validate_ipaq(
    prefix: str, # prefix for the dataset
    df: pl.DataFrame
    ) -> pb.Validate:

    validation = (
        pb.Validate(
            data=df,
        )
        .col_vals_eq(
            columns=f"{prefix}_IPAQ_VIG_MINS",
            value=pb.col("check"),
            pre=check_total_mins(f"{prefix}_IPAQ_VIG_HPD", f"{prefix}_IPAQ_VIG_MPD"),
            brief="Check total mins/day equals `HPD*60 + MPD`",
            na_pass=True,
        )
        .col_vals_eq(
            columns=f"{prefix}_IPAQ_MOD_MINS",
            value=pb.col("check"),
            pre=check_total_mins(f"{prefix}_IPAQ_MOD_HPD", f"{prefix}_IPAQ_MOD_MPD"),
            brief="Check total mins/day equals `HPD*60 + MPD`",
            na_pass=True,
        )
        .col_vals_eq(
            columns=f"{prefix}_IPAQ_WALK_MINS",
            value=pb.col("check"),
            pre=check_total_mins(f"{prefix}_IPAQ_WALK_HPD", f"{prefix}_IPAQ_WALK_MPD"),
            brief="Check total mins/day equals `HPD*60 + MPD`",
            na_pass=True,
        )
        # .col_vals_eq(
        #     columns=f"{prefix}_IPAQ_VIG_MET",
        #     value=pb.col("check"),
        #     pre=check_met(f"{prefix}_IPAQ_VIG_MINS", f"{prefix}_IPAQ_VIG_D", f"{prefix}_IPAQ_VIG_MET", factor=8),
        # )
        # .col_vals_eq(
        #     columns=f"{prefix}_IPAQ_MOD_MET",
        #     value=pb.col("check"),
        #     pre=check_met(f"{prefix}_IPAQ_MOD_MINS", f"{prefix}_IPAQ_MOD_D", f"{prefix}_IPAQ_MOD_MET", factor=4),
        # )
        # .col_vals_eq(
        #     columns=f"{prefix}_IPAQ_WALK_MET",
        #     value=pb.col("check"),
        #     pre=check_met(f"{prefix}_IPAQ_WALK_MINS", f"{prefix}_IPAQ_WALK_D", f"{prefix}_IPAQ_WALK_MET", factor=3.3),
        # )
        # .col_vals_eq(
        #     columns=f"{prefix}_IPAQ_TOT_MET",
        #     value=pb.col("check"),
        #     pre=check_tot_met([f"{prefix}_IPAQ_VIG_MET", f"{prefix}_IPAQ_MOD_MET", f"{prefix}_IPAQ_WALK_MET"], f"{prefix}_IPAQ_TOT_MET"),
        #     brief="Check `TOT_MET` equals the sum of `VIG_MET`, `MOD_MET`, and `WALK_MET`"
        # )
        # .col_vals_between(
        #     columns=f"{prefix}_IPAQ_VIG_D",
        #     left=1,
        #     right=7,
        #     segments=(f"{prefix}_IPAQ_VIG_W", 1),
        # )
        # .col_vals_between(
        #     columns=f"{prefix}_IPAQ_VIG_HPD", 
        #     left=0,
        #     right=18, # unrealistic to do more than 18 hours of exercise per day (even that is a stretch!!)
        #     segments=(f"{prefix}_IPAQ_VIG_W", 1),
        # )
        # .col_vals_between(
        #     columns=f"{prefix}_IPAQ_VIG_MPD", 
        #     left=0,
        #     right=59,
        #     segments=(f"{prefix}_IPAQ_VIG_W", 1),
        # )
        # .col_vals_between(
        #     columns=f"{prefix}_IPAQ_VIG_MINS", 
        #     left=0,
        #     right=180, # total mins per category is capped at 180 mins
        #     segments=(f"{prefix}_IPAQ_VIG_W", 1),
        # )
        # .col_vals_eq(
        #     columns=[f"{prefix}_IPAQ_VIG_D", f"{prefix}_IPAQ_VIG_HPD", f"{prefix}_IPAQ_VIG_MPD", f"{prefix}_IPAQ_VIG_MINS", f"{prefix}_IPAQ_VIG_MET"], 
        #     value=0,
        #     segments=(f"{prefix}_IPAQ_VIG_W", 0)
        # )
        # .col_vals_null(
        #     columns=[f"{prefix}_IPAQ_VIG_D", f"{prefix}_IPAQ_VIG_HPD", f"{prefix}_IPAQ_VIG_MPD", f"{prefix}_IPAQ_VIG_MINS", f"{prefix}_IPAQ_VIG_MET"],
        #     pre=lambda df: df.with_columns(pl.col(f"{prefix}_IPAQ_VIG_W").fill_null(-1)), # Pointblank doesn't seem to like segmenting values with null, so transform null to -1 and segment y that
        #     segments=(f"{prefix}_IPAQ_VIG_W", -1)
        # )
        # .col_vals_between(
        #     columns=f"{prefix}_IPAQ_MOD_D",
        #     left=1,
        #     right=7,
        #     segments=(f"{prefix}_IPAQ_MOD_W", 1),
        # )
        # .col_vals_between(
        #     columns=f"{prefix}_IPAQ_MOD_HPD", 
        #     left=0,
        #     right=18, # unrealistic to do more than 18 hours of exercise per day (even that is a stretch!!)
        #     segments=(f"{prefix}_IPAQ_MOD_W", 1),
        # )
        # .col_vals_between(
        #     columns=f"{prefix}_IPAQ_MOD_MPD", 
        #     left=0,
        #     right=59,
        #     segments=(f"{prefix}_IPAQ_MOD_W", 1),
        # )
        # .col_vals_between(
        #     columns=f"{prefix}_IPAQ_MOD_MINS", 
        #     left=0,
        #     right=180, # total mins per category is capped at 180 mins
        #     segments=(f"{prefix}_IPAQ_MOD_W", 1),
        # )
        # .col_vals_eq(
        #     columns=[f"{prefix}_IPAQ_MOD_D", f"{prefix}_IPAQ_MOD_HPD", f"{prefix}_IPAQ_MOD_MPD", f"{prefix}_IPAQ_MOD_MINS", f"{prefix}_IPAQ_MOD_MET"], 
        #     value=0,
        #     segments=(f"{prefix}_IPAQ_MOD_W", 0)
        # )
        # .col_vals_null(
        #     columns=[f"{prefix}_IPAQ_MOD_D", f"{prefix}_IPAQ_MOD_HPD", f"{prefix}_IPAQ_MOD_MPD", f"{prefix}_IPAQ_MOD_MINS", f"{prefix}_IPAQ_MOD_MET"],
        #     pre=lambda df: df.with_columns(pl.col(f"{prefix}_IPAQ_MOD_W").fill_null(-1)), # Pointblank doesn't seem to like segmenting values with null, so transform null to -1 and segment y that
        #     segments=(f"{prefix}_IPAQ_MOD_W", -1)
        # )
        # .col_vals_between(
        #     columns=f"{prefix}_IPAQ_WALK_D",
        #     left=1,
        #     right=7,
        #     segments=(f"{prefix}_IPAQ_WALK_W", 1),
        # )
        # .col_vals_between(
        #     columns=f"{prefix}_IPAQ_WALK_HPD", 
        #     left=0,
        #     right=18, # unrealistic to do more than 18 hours of exercise per day (even that is a stretch!!)
        #     segments=(f"{prefix}_IPAQ_WALK_W", 1),
        # )
        # .col_vals_between(
        #     columns=f"{prefix}_IPAQ_WALK_MPD", 
        #     left=0,
        #     right=59,
        #     segments=(f"{prefix}_IPAQ_WALK_W", 1),
        # )
        # .col_vals_between(
        #     columns=f"{prefix}_IPAQ_WALK_MINS", 
        #     left=0,
        #     right=180, # total mins per category is capped at 180 mins
        #     segments=(f"{prefix}_IPAQ_WALK_W", 1),
        # )
        # .col_vals_eq(
        #     columns=[f"{prefix}_IPAQ_WALK_D", f"{prefix}_IPAQ_WALK_HPD", f"{prefix}_IPAQ_WALK_MPD", f"{prefix}_IPAQ_WALK_MINS", f"{prefix}_IPAQ_WALK_MET"], 
        #     value=0,
        #     segments=(f"{prefix}_IPAQ_WALK_W", 0)
        # )
        # .col_vals_null(
        #     columns=[f"{prefix}_IPAQ_WALK_D", f"{prefix}_IPAQ_WALK_HPD", f"{prefix}_IPAQ_WALK_MPD", f"{prefix}_IPAQ_WALK_MINS", f"{prefix}_IPAQ_WALK_MET"],
        #     pre=lambda df: df.with_columns(pl.col(f"{prefix}_IPAQ_WALK_W").fill_null(-1)), # Pointblank doesn't seem to like segmenting values with null, so transform null to -1 and segment y that
        #     segments=(f"{prefix}_IPAQ_WALK_W", -1)
        # )
        .col_vals_eq(
            columns=f"{prefix}_IPAQ_CAT",
            value=pb.col("check"),
            pre=check_ipaq_cat([f"{prefix}_IPAQ_VIG_MET", f"{prefix}_IPAQ_MOD_MET", f"{prefix}_IPAQ_WALK_MET"], f"{prefix}_IPAQ_CAT"),
            brief="Check `IPAQ_CAT` is correctly calculated."
        )
    )

    return validation

In [ ]:
validation = validate_ipaq(prefix="G222", df=df_g222).interrogate()